#01_EDA

In [21]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib


import sklearn
import xgboost
from xgboost import XGBRegressor


from sklearn.model_selection import KFold

import random as rn

RANDOM_SEED = 2025
np.random.seed(RANDOM_SEED)
rn.seed(RANDOM_SEED)



from datetime import datetime

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [22]:
# 한글 폰트 관련
# 1. 나눔 폰트 설치
!apt-get -y install fonts-nanum

# 2. 런타임 재시작 없이 폰트 강제 등록
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib as mpl

# 3. 폰트 경로 직접 지정
font_path = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"
fm.fontManager.addfont(font_path)
nanum_font = fm.FontProperties(fname=font_path).get_name()

# 4. matplotlib 기본 설정에 강제로 반영
mpl.rcParams['font.family'] = nanum_font
mpl.rcParams['axes.unicode_minus'] = False


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [23]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
%cd /content/drive/MyDrive/dacon_2025_Power_usage_prediction_AI_github/2025dacon_powerU_github

/content/drive/MyDrive/dacon_2025_Power_usage_prediction_AI_github/2025dacon_powerU_github


In [25]:
!git config --global user.name "SonHyesang0914"
!git config --global user.email "hyesang0914@gmail.com"

##02-7. 건물별 예상 휴일 추가 (개인 판단)

In [26]:
train_preprocessed_path = "/content/drive/MyDrive/dacon_2025_Power_usage_prediction_AI/dacon_data/train_preprocessed.csv"
test_preprocessed_path = "/content/drive/MyDrive/dacon_2025_Power_usage_prediction_AI/dacon_data/test_preprocessed.csv"
train_preprocessed = pd.read_csv(train_preprocessed_path)
test_preprocessed = pd.read_csv(test_preprocessed_path)

In [27]:
# date_time을 datetime으로 변환
train_preprocessed['date_time'] = pd.to_datetime(train_preprocessed['date_time'])
test_preprocessed['date_time'] = pd.to_datetime(test_preprocessed['date_time'])


In [28]:
# 1) 우선 모든 행을 0으로 초기화
train_preprocessed['holiday_by_building'] = 0
test_preprocessed['holiday_by_building']  = 0

# 2) 휴일로 판단한 날짜 목록 정의
#    형식: {건물번호: [(월, 일), (월, 일), ...], ...}
manual_holidays_test = {
    3: [(8, 25)],
    5: [(8, 25)],
    6: [(8, 25)],
    8: [(8, 25)],
    12: [(8, 25)],
    13: [(8, 25)],
    14: [(8, 25)],
    15: [(8, 25)],
    16: [(8, 25)],
    17: [(8, 25)],
    20: [(8, 25)],
    21: [(8, 25)],
    22: [(8, 25)],
    23: [(8, 25)],
    24: [(8, 25)],
    27: [(8, 25)],
    29: [(8, 25)],
    32: [(8, 26)],
    37: [(8, 25)],
    38: [(8, 25)],
    39: [(8, 25)],
    40: [(8, 25)],
    42: [(8, 25)],
    43: [(8, 25)],
    44: [(8, 25)],
    46: [(8, 25)],
    47: [(8, 25)],
    48: [(8, 25)],
    49: [(8, 25)],
    50: [(8, 25)],
    51: [(8, 25)],
    52: [(8, 25)],
    53: [(8, 25)],
    55: [(8, 25)],
    59: [(8, 25)],
    60: [(8, 25)],
    62: [(8, 25)],
    63: [(8, 25)],
    64: [(8, 25)],
    66: [(8, 25)],
    67: [(8, 25)],
    68: [(8, 25)],
    69: [(8, 25)],
    72: [(8, 25)],
    74: [(8, 26)],
    75: [(8, 25)],
    80: [(8, 25)],
    83: [(8, 25)],
    86: [(8, 25)],
    87: [(8, 25)],
    90: [(8, 25)],
    94: [(8, 25)],
}
# 2) 휴일로 판단한 날짜 목록 정의
#    형식: {건물번호: [(월, 일), (월, 일), ...], ...}
manual_holidays_train = {
    2: [(6, 1), (6, 8), (6, 15), (6, 22), (6, 29), (7, 6), (7, 13), (7, 20), (7, 27), (8, 3), (8, 10), (8, 17), (8, 24)],
    3: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    5: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    6: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    7: [(7, 7), (7, 8)],
    8: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    12: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    13: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    14: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    15: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    16: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    17: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    18: [(6, 11), (7, 17), (8, 8)],
    19: [(6, 10), (7, 8), (8, 19)],
    20: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    21: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    22: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    23: [(6, 1), (6, 2), (6, 6), (6, 7), (6, 8), (6, 9), (6, 15), (6, 16), (6, 21), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 12), (7, 13), (7, 14), (7, 19), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 16), (8, 17), (8, 18), (8, 23), (8, 24)],
    24: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    27: [(6, 9), (6, 23), (7, 14), (7, 28), (8, 11)],
    29: [(6, 10), (6, 23), (7, 10), (7, 28), (8, 10)],
    32: [(6, 10), (6, 24), (7, 8), (7, 22), (8, 12)],
    37: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    38: [(6, 1), (6, 2), (6, 6), (6, 7), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    39: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    40: [(6, 9), (6, 23), (7, 14), (7, 28), (8, 11)],
    42: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    43: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    44: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    45: [(6, 10), (7, 8), (8, 19)],
    46: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    47: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    48: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    49: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 22), (8, 24)],
    50: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    51: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    52: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 28), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    53: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    54: [(6, 17), (7, 1), (8, 19)],
    55: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    59: [(6, 9), (6, 23), (7, 14), (7, 28), (8, 11)],
    60: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    62: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    63: [(6, 9), (6, 23), (7, 14), (7, 28), (8, 11)],
    64: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    66: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    67: [(6, 1), (6, 2), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    68: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    69: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    72: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    74: [(6, 17), (7, 1)],
    75: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    79: [(6, 17), (7, 1), (8, 19)],
    80: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    83: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    86: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    87: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    90: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    94: [(6, 1), (6, 2), (6, 6), (6, 7), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (8, 10), (8, 11), (8, 15), (8, 16), (8, 17), (8, 18), (8, 24)],
    95: [(7, 8), (7, 5)]
}


# 3) train에 반영
for bld, dates in manual_holidays_train.items():
    for mm, dd in dates:
        mask = (train_preprocessed['building_num'] == bld) & (train_preprocessed['month'] == mm) & (train_preprocessed['day'] == dd)
        train_preprocessed.loc[mask, 'holiday_by_building'] = 1

# 4) test에도 동일 규칙 반영
for bld, dates in manual_holidays_test.items():
    for mm, dd in dates:
        mask = (test_preprocessed['building_num'] == bld) & (test_preprocessed['month'] == mm) & (test_preprocessed['day'] == dd)
        test_preprocessed.loc[mask, 'holiday_by_building'] = 1


In [29]:
for day in range(13, 32):
    mask = (train_preprocessed['building_num'] == 7) & (train_preprocessed['month'] == 7) & (train_preprocessed['day'] == day)
    train_preprocessed.loc[mask, 'holiday_by_building'] = 1


In [30]:
for day in range(1, 6):
    mask = (train_preprocessed['building_num'] == 7) & (train_preprocessed['month'] == 8) & (train_preprocessed['day'] == day)
    train_preprocessed.loc[mask, 'holiday_by_building'] = 1



In [ ]:
for day in range(25, 31):

    mask_test = (test_preprocessed['building_num'] == 10) & (test_preprocessed['month'] == 8) & (test_preprocessed['day'] == day)
    test_preprocessed.loc[mask_test, 'holiday_by_building'] = 1

In [ ]:
for day in range(1, 31):
    mask = (train_preprocessed['building_num'] == 10) & (train_preprocessed['month'] == 6) & (train_preprocessed['day'] == day)
    train_preprocessed.loc[mask, 'holiday_by_building'] = 1

In [ ]:
for day in range(1, 4):
    mask = (train_preprocessed['building_num'] == 10) & (train_preprocessed['month'] == 7) & (train_preprocessed['day'] == day)
    train_preprocessed.loc[mask, 'holiday_by_building'] = 1

In [31]:
for day in range(1, 31):
    mask = (train_preprocessed['building_num'] == 11) & (train_preprocessed['month'] == 6) & (train_preprocessed['day'] == day)
    train_preprocessed.loc[mask, 'holiday_by_building'] = 1



In [32]:
for day in range(1, 20):
    mask = (train_preprocessed['building_num'] == 11) & (train_preprocessed['month'] == 7) & (train_preprocessed['day'] == day)
    train_preprocessed.loc[mask, 'holiday_by_building'] = 1



In [33]:
for day in range(27, 32):
    mask = (train_preprocessed['building_num'] == 94) & (train_preprocessed['month'] == 7) & (train_preprocessed['day'] == day)
    train_preprocessed.loc[mask, 'holiday_by_building'] = 1


In [34]:
for day in range(1, 5):
    mask = (train_preprocessed['building_num'] == 94) & (train_preprocessed['month'] == 8) & (train_preprocessed['day'] == day)
    train_preprocessed.loc[mask, 'holiday_by_building'] = 1

##02-7. 성수기 여부 (7/10- ~8/15 가정)

In [ ]:
# -----------------------------
# 1) date_time을 datetime으로 변환
# -----------------------------
for df in [train_preprocessed, test_preprocessed]:
    if "date_time" in df.columns:
        df["date_time"] = pd.to_datetime(df["date_time"])

# -----------------------------
# 2) 성수기 feature 추가
# -----------------------------
def add_peak_summer_feature(df):
    # 2024년 성수기 기간
    start = pd.Timestamp("2024-07-10")
    end   = pd.Timestamp("2024-08-15")

    df["is_peak_summer"] = df["date_time"].between(start, end).astype(int)
    return df

train_preprocessed = add_peak_summer_feature(train_preprocessed)
test_preprocessed = add_peak_summer_feature(test_preprocessed)

# -----------------------------
# 3) 확인
# -----------------------------
print(train_df[["date_time", "is_peak_summer"]].head(40))


In [36]:
# 전처리 끝난 DataFrame 이름이 train_final, test_final 이라고 가정
out_dir = "/content/drive/MyDrive/dacon_2025_Power_usage_prediction_AI/dacon_data/"

# train 저장
train_preprocessed.to_csv(out_dir + "train_preprocessed4.csv", index=False)

# test 저장
test_preprocessed.to_csv(out_dir + "test_preprocessed4.csv", index=False)

In [37]:
!git status
!git add 02_EDAandPreprocessing_2.ipynb
!git commit -m "2차 전처리-4"
!git push origin main

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   01_EDA.ipynb
	modified:   02_EDAandPreprocessing.ipynb
	modified:   02_EDAandPreprocessing_2.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.gitignore
	03_Modeling_test.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
[main 20bacfc] 2차 전처리-2
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite 02_EDAandPreprocessing_2.ipynb (69%)
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 8.70 KiB | 1.09 MiB/s, done.
Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/SonHyesang09